# White House visitor logs

### Import Python tools and Jupyter configuration

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import datetime as dt
import altair as alt
import numpy as np

In [3]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()
pd.options.display.max_colwidth = None

### Data dictionary

In [4]:
# "https://obamawhitehouse.archives.gov/files/disclosures/visitors/WhiteHouse-WAVES-Key-1209.txt"

In [5]:
today = dt.date.today().strftime("%m-%d-%Y")

In [6]:
df = pd.read_csv(
    "data/processed/log_latest.csv",
    parse_dates=[
        "appt_made_date",
        "appt_start_date",
        "appt_end_date",
        "appt_cancel_date",
        "releasedate",
    ],
)

In [7]:
df.sort_values(["appt_made_date"], ascending=False).head()

,namelast,namefirst,namemid,uin,bdgnbr,access_type,toa,poa,tod,pod,appt_made_date,appt_start_date,appt_end_date,appt_cancel_date,total_people,last_updatedby,post,lastentrydate,terminal_suffix,visitee_namelast,visitee_namefirst,meeting_loc,meeting_room,caller_name_last,caller_name_first,caller_room,description,releasedate
17257,COFIELD,NATALIE,M,U32379,182338.0,VA,9/29/21 13:49,NAN,NAN,A05,2021-09-29 12:54:00,2021-09-29,2021-09-29 11:59:00,NaT,1.0,NP,WIN,9/29/2021 12:54,NP,PAGE,NIA,WH,WW 206/20,PAGE,NIA,NaN,NAN,2022-01-07
16581,ROZENSHTEIN,JOSEPH,A,U32371,176418.0,VA,9/29/21 15:46,NAN,NAN,B04,2021-09-29 12:15:00,2021-09-29,2021-09-29 11:59:00,NaT,1.0,SE,WIN,9/29/2021 12:15,SE,DECESARO,JENNIFER,OEOB,367,EISEMANN,SARA,NaN,NAN,2022-01-07
15241,HANNA,CORY,M,U32370,183123.0,VA,9/29/21 17:23,NAN,NAN,B04,2021-09-29 12:10:00,2021-09-29,2021-09-29 11:59:00,NaT,1.0,PD,WIN,9/29/2021 12:10,PD,DIEN,PETER,OEOB,386,DIEN,PETER,NaN,NAN,2022-01-07
14849,KRAUT,JAMIE,N,U32302,183101.0,VA,9/29/21 15:55,NAN,NAN,A05,2021-09-29 12:04:00,2021-09-29,2021-09-29 11:59:00,NaT,12.0,AD,WIN,9/29/2021 12:04,AD,TAYLOR,BEN,OEOB,230A,DIETRICH,ANNA,NaN,NAN,2022-01-07
17022,EMERSON,EGYPT,W,U32366,NaN,VA,9/29/21 14:45,NAN,NAN,NAN,2021-09-29 11:27:00,2021-09-29,2021-09-29 11:59:00,NaT,1.0,HQ,WIN,9/29/2021 11:27,HQ,QURESHI,HOOR,OEOB,115,QURESHI,HOOR,NaN,NAN,2022-01-07


### Buildings

In [8]:
df.meeting_loc.value_counts()

OEOB    8495
WH      7292
NEOB    1007
EEOB     837
VPR      835
NAN      737
Name: meeting_loc, dtype: int64

### Rooms 

In [9]:
df.meeting_room.value_counts().head()

SOUTH LAWN    1254
EW206          884
445            846
RES - STAT     783
NAN            737
Name: meeting_room, dtype: int64

### Who's visited the Oval?

In [10]:
len(df[df["meeting_room"].str.contains("OVAL")])

387

In [11]:
df[df["meeting_room"].str.contains("OVAL")].head()

,namelast,namefirst,namemid,uin,bdgnbr,access_type,toa,poa,tod,pod,appt_made_date,appt_start_date,appt_end_date,appt_cancel_date,total_people,last_updatedby,post,lastentrydate,terminal_suffix,visitee_namelast,visitee_namefirst,meeting_loc,meeting_room,caller_name_last,caller_name_first,caller_room,description,releasedate
257,JANKELOWHARRIS,JASMINE,A,U25091,NaN,VA,5/20/2021 9:50,SOS,NAN,NAN,2021-05-17 19:01:00,2021-05-20 09:45:00,2021-05-20 23:59:00,NaT,12.0,MD,WIN,5/17/2021 19:01,MD,NAN,POTUS,WH,OVAL OFFIC,DURAN,MANUELITA,NaN,NAN,2021-08-30
259,GATO,MIDORI,N,U25091,NaN,VA,5/20/2021 9:50,SOS,NAN,NAN,2021-05-17 19:01:00,2021-05-20 09:45:00,2021-05-20 23:59:00,NaT,12.0,MD,WIN,5/17/2021 19:01,MD,NAN,POTUS,WH,OVAL OFFIC,DURAN,MANUELITA,NaN,NAN,2021-08-30
260,DANZIGER,CHARLES,T,U25091,NaN,VA,5/20/2021 9:50,SOS,NAN,NAN,2021-05-17 19:01:00,2021-05-20 09:45:00,2021-05-20 23:59:00,NaT,12.0,MD,WIN,5/17/2021 19:01,MD,NAN,POTUS,WH,OVAL OFFIC,DURAN,MANUELITA,NaN,NAN,2021-08-30
261,BROOKS,TROYAL,G,U25091,NaN,VA,5/20/2021 9:50,SOS,NAN,NAN,2021-05-17 19:01:00,2021-05-20 09:45:00,2021-05-20 23:59:00,NaT,12.0,MD,WIN,5/17/2021 19:01,MD,NAN,POTUS,WH,OVAL OFFIC,DURAN,MANUELITA,NaN,NAN,2021-08-30
262,BAEZ,JOAN,C,U25091,NaN,VA,5/20/2021 9:50,SOS,NAN,NAN,2021-05-17 19:01:00,2021-05-20 09:45:00,2021-05-20 23:59:00,NaT,12.0,MD,WIN,5/17/2021 19:01,MD,NAN,POTUS,WH,OVAL OFFIC,DURAN,MANUELITA,NaN,NAN,2021-08-30


### Where meeting room is ...

In [12]:
df[df["meeting_room"].str.contains("EW206")].head()

,namelast,namefirst,namemid,uin,bdgnbr,access_type,toa,poa,tod,pod,appt_made_date,appt_start_date,appt_end_date,appt_cancel_date,total_people,last_updatedby,post,lastentrydate,terminal_suffix,visitee_namelast,visitee_namefirst,meeting_loc,meeting_room,caller_name_last,caller_name_first,caller_room,description,releasedate
2,WILCOXSON,MARTHA,L,U25238,NaN,VA,NAN,NAN,NAN,NAN,2021-05-20 09:06:00,2021-05-21 10:00:00,2021-05-21 23:59:00,NaT,31.0,DV,WIN,5/20/2021 9:20,DV,NAN,POTUS,WH,EW206,VIA,DANIEL,NaN,NAN,2021-08-30
3,WILCOXSON,ANTHONY,L,U25238,NaN,VA,NAN,NAN,NAN,NAN,2021-05-20 09:06:00,2021-05-21 10:00:00,2021-05-21 23:59:00,NaT,31.0,DV,WIN,5/20/2021 9:06,DV,NAN,POTUS,WH,EW206,VIA,DANIEL,NaN,NAN,2021-08-30
4,WHITMAN,RALPH,E,U25238,NaN,VA,NAN,NAN,NAN,NAN,2021-05-20 09:06:00,2021-05-21 10:00:00,2021-05-21 23:59:00,NaT,31.0,DV,WIN,5/20/2021 9:06,DV,NAN,POTUS,WH,EW206,VIA,DANIEL,NaN,NAN,2021-08-30
10,SOLHJEM,THOMAS,L,U25238,NaN,VA,NAN,NAN,NAN,NAN,2021-05-20 09:06:00,2021-05-21 10:00:00,2021-05-21 23:59:00,NaT,31.0,DV,WIN,5/20/2021 9:06,DV,NAN,POTUS,WH,EW206,VIA,DANIEL,NaN,NAN,2021-08-30
11,SIMPSON,MERLE,W,U25238,179329.0,VA,5/21/2021 10:05,B0401,NAN,NAN,2021-05-20 09:06:00,2021-05-21 10:00:00,2021-05-21 23:59:00,NaT,31.0,DV,WIN,5/20/2021 9:06,DV,NAN,POTUS,WH,EW206,VIA,DANIEL,NaN,NAN,2021-08-30


### Where 'visitee' is explicity the Vice President

In [13]:
len(df[df["visitee_namefirst"].str.contains("VPOTUS")])

98

In [14]:
df[df["visitee_namefirst"].str.contains("VPOTUS")].to_csv(
    "data/processed/vpotus_visitee.csv", index=False
)